In [16]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.datasets import cifar10, imdb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import scipy

def preprocess_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    # Wybierz klasy 3 i 5
    mask_train = np.logical_or(y_train == 3, y_train == 5).ravel()
    X_train = X_train[mask_train]
    y_train = y_train[mask_train]

    mask_test = np.logical_or(y_test == 3, y_test == 5).ravel()
    X_test = X_test[mask_test]
    y_test = y_test[mask_test]
    # Zmiana etykiet na binarne
    y_train = np.where(y_train == 3, 0, 1)
    y_test = np.where(y_test == 3, 0, 1)
    return X_train, y_train, X_test, y_test

def preprocess_imdb():
    (X_train, y_train), (X_test, y_test) = imdb.load_data(path="imdb.npz",num_words=None,skip_top=0, maxlen=None,seed=1,start_char=1,oov_char=2,index_from=3)
    # Odwróć indeks słownika IMDB, aby uzyskać mapowanie z indeksu na słowo
    word_index = imdb.get_word_index()
    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

    # Dekoduj recenzje do tekstów
    decoded_train = [" ".join([reverse_word_index.get(i - 3, "?") for i in review]) for review in X_train] #Dlaczego i - 3? - trzy pierwsze indeksy są zarezerwowane dla wartości specjalnych: start sekwencji, nieznane słowo i koniec sekwencji, więc indeksy rzeczywistych słów w słowniku są przesunięte o 3.
    decoded_test = [" ".join([reverse_word_index.get(i - 3, "?") for i in review]) for review in X_test]

    vectorizer = TfidfVectorizer(max_features=2500)
    X_train = vectorizer.fit_transform(decoded_train)
    X_test = vectorizer.transform(decoded_test)

    return X_train, y_train, X_test, y_test

def preprocess_breast_cancer():
    data = load_breast_cancer()
    X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)
    return X_train, y_train, X_test, y_test


def split_labeled_unlabeled(X, y, labeled_size):
    X_labeled, X_unlabeled, y_labeled, _ = train_test_split(X, y, test_size=1-labeled_size, random_state=42, stratify=y)
    return X_labeled, y_labeled, X_unlabeled

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(y_pred)
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred)
    }

def preprocess_data(dataset_name):
    if dataset_name == 'cifar10':
        return preprocess_cifar10()
    elif dataset_name == 'imdb':
        return preprocess_imdb()
    elif dataset_name == 'breast_cancer':
        return preprocess_breast_cancer()

def train_and_evaluate(dataset_name, model, X_train, y_train, X_unlabeled, X_test, y_test):
    print(f"Training model on {dataset_name}")
          
    if dataset_name == 'cifar10':
        y_train = y_train.ravel()
        y_test = y_test.ravel()
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_unlabeled = X_unlabeled.reshape(X_unlabeled.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)

        # Konwersja rzadkich macierzy do gęstych tablic dla danych IMDB
    elif dataset_name == 'imdb':
        X_train = X_train.toarray() if scipy.sparse.issparse(X_train) else X_train
        X_unlabeled = X_unlabeled.toarray() if scipy.sparse.issparse(X_unlabeled) else X_unlabeled
        X_test = X_test.toarray() if scipy.sparse.issparse(X_test) else X_test

    print(f"X_train shape: {X_train.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_train shape: {y_train.shape}")
    start_time = time.time()
    model.fit(X=X_train, y=y_train, unlabeled_X=X_unlabeled)
    training_time = time.time() - start_time
    results = evaluate_model(model, X_test, y_test)
    return results, training_time

# <font color='green'>Tri-training</font>

### Hiperparametry modelu Tri-Training

1. **base_estimator**: Pierwszy podstawowy uczeń w TriTraining.
   - Możliwe wartości: dowolny klasyfikator zgodny z API `scikit-learn`.
   
   
2. **base_estimator_2**: Drugi podstawowy uczeń w TriTraining.
   - Możliwe wartości: dowolny klasyfikator zgodny z API `scikit-learn`.


3. **base_estimator_3**: Trzeci podstawowy uczeń w TriTraining.
   - Możliwe wartości: dowolny klasyfikator zgodny z API `scikit-learn`.

In [71]:
from LAMDA_SSL.Algorithm.Classification.Tri_Training import Tri_Training
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
datasets = ['cifar10', 'imdb', 'breast_cancer']
labeled_size = 0.3
base_estimators = [KNeighborsClassifier(n_neighbors=7), RandomForestClassifier(), LinearSVC(max_iter=10**4)]
models = {dataset: Tri_Training(base_estimator=base_estimators[0], base_estimator_2=base_estimators[1], base_estimator_3=base_estimators[2]) for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing cifar10
X_labeled shape: (3000, 32, 32, 3), X_unlabeled shape: (7000, 32, 32, 3)
y_labeled shape: (3000, 1)
Training model on cifar10
X_train shape: (3000, 3072), X_unlabeled shape: (7000, 3072)
y_train shape: (3000,)


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:1201: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:1201: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:1201: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


cifar10 Results: {'accuracy': 0.6355, 'precision': 0.66073546856465, 'recall': 0.557, 'f1_score': 0.6044492674986436}
Training Time cifar10: 970.4850695133209

Processing imdb
X_labeled shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_labeled shape: (7500,)
Training model on imdb
X_train shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_train shape: (7500,)
imdb Results: {'accuracy': 0.8428, 'precision': 0.8402954256670903, 'recall': 0.84648, 'f1_score': 0.8433763749402199}
Training Time imdb: 133.80731654167175

Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:1201: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:1201: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


breast_cancer Results: {'accuracy': 0.9385964912280702, 'precision': 0.9102564102564102, 'recall': 1.0, 'f1_score': 0.953020134228188}
Training Time breast_cancer: 0.9029445648193359



c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:1201: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


#  <font color='green'>ASSEMBLE</font>

### Hiperparametry

1. **base_estimator** - Podstawowy klasyfikator używany w uczeniu zespołowym. Dowolny klasyfikator zgodny z interfejsem sklearn (np. DecisionTreeClassifier, KNeighborsClassifier).


2. **T** - Liczba podstawowych klasyfikatorów, równocześnie jest to liczba iteracji.
    Możliwe wartości: Liczby całkowite większe od 0.


3. **alpha** - Waga każdej próbki przy aktualizacji rozkładu próbkowania.
    Możliwe wartości: Liczby rzeczywiste większe od 0.


4. **beta** - Używany do inicjalizacji rozkładu próbkowania danych oznaczonych i nieoznaczonych.
    Możliwe wartości: Liczby rzeczywiste w przedziale [0, 1].


In [8]:
from LAMDA_SSL.Algorithm.Classification.Assemble import Assemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
#datasets = ['cifar10','imdb', 'breast_cancer']
datasets = ['imdb', 'breast_cancer']
labeled_size = 0.3
base_estimators = [KNeighborsClassifier(n_neighbors=7), RandomForestClassifier(), LinearSVC(max_iter=10**4)]
T=10
models = {dataset: Assemble(T=T,base_estimator=base_estimators[1]) for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing imdb
X_labeled shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_labeled shape: (7500,)
Training model on imdb
X_train shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_train shape: (7500,)
imdb Results: {'accuracy': 0.8132, 'precision': 0.8071069971760276, 'recall': 0.82312, 'f1_score': 0.8150348542458808}
Training Time imdb: 460.310693025589

Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
breast_cancer Results: {'accuracy': 0.956140350877193, 'precision': 0.9342105263157895, 'recall': 1.0, 'f1_score': 0.9659863945578232}
Training Time breast_cancer: 3.2257578372955322



#  <font color='green'>Semiboost</font>

### Hiperparametry 

1. **base_estimator**
   - Opis: Bazowy nadzorowany uczeń używany w algorytmie.
   - Możliwe wartości: Dowolny klasyfikator zgodny z interfejsem scikit-learn.


2. **n_neighbors**
   - Opis: Liczba sąsiadów w funkcji kernela 'knn'.
   - Możliwe wartości: Dowolna liczba całkowita większa od 0.


3. **n_jobs**
   - Opis: Liczba równoległych zadań w funkcji kernela 'knn'.
   - Możliwe wartości: Dowolna liczba całkowita lub None.


4. **T**
   - Opis: Liczba bazowych uczących się, równa także liczbie iteracji.
   - Możliwe wartości: Dowolna liczba całkowita większa od 0.


5. **sample_percent**
   - Opis: Procent próbek pobieranych w każdej iteracji jako proporcja pozostałych nieoznakowanych próbek.
   - Możliwe wartości: Liczba zmiennoprzecinkowa od 0 do 1.


6. **sigma_percentile**
   - Opis: Parametr skali używany w kernelu 'rbf'.
   - Możliwe wartości: Dowolna liczba zmiennoprzecinkowa większa od 0.


7. **similarity_kernel**
   - Opis: Typ kernela, 'rbf', 'knn' lub funkcja.
   - Możliwe wartości: 'rbf', 'knn', 'linear' lub funkcja.


8. **gamma**
   - Opis: Wartość gamma dla kernela 'rbf'.
   - Możliwe wartości: Dowolna liczba zmiennoprzecinkowa większa od 0.

In [10]:
from LAMDA_SSL.Algorithm.Classification.SemiBoost import SemiBoost
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
#datasets = ['cifar10','imdb', 'breast_cancer']
datasets = ['imdb', 'breast_cancer']
labeled_size = 0.3
T=10
base_estimators = [KNeighborsClassifier(n_neighbors=7), RandomForestClassifier(), LinearSVC(max_iter=10**4)]
models = {dataset: SemiBoost(T=T,base_estimator=base_estimators[1]) for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing imdb
X_labeled shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_labeled shape: (7500,)
Training model on imdb
X_train shape: (7500, 2500), X_unlabeled shape: (17500, 2500)
y_train shape: (7500,)
imdb Results: {'accuracy': 0.744, 'precision': 0.9117170626349892, 'recall': 0.54032, 'f1_score': 0.6785211975085392}
Training Time imdb: 4773.320457458496

Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
breast_cancer Results: {'accuracy': 0.9649122807017544, 'precision': 0.958904109589041, 'recall': 0.9859154929577465, 'f1_score': 0.9722222222222222}
Training Time breast_cancer: 2.3200249671936035



# <font color='green'>LapSVM</font>

### Hiperparametry

1. **distance_function**
    - Funkcja odległości używana do budowy grafu. Ten parametr jest ważny, gdy neighbor_mode jest None.
    - Możliwe wartości: Zależne od implementacji, ale często obejmują funkcje takie jak 'knn', 'linear', 'rbf' oraz dowolne funkcje zdefiniowane przez użytkownika.


2. **gamma_d**
    - Opis: Parametr kernela związany z distance_function.
    - Możliwe wartości: Dowolna liczba rzeczywista, zależna od specyficznej funkcji odległości.


3. **neighbor_mode**
    - Opis: Tryb krawędzi po zbudowaniu modelu grafu przez k-najbliższych sąsiadów. Opcje to 'connectivity' (zwraca macierz 0-1) i 'distance' (zwraca macierz odległości).
    - Możliwe wartości: 'connectivity', 'distance'.


4. **t**
    - Opis: Parametr używany w obliczeniach, szczególnie gdy distance_function jest 'knn'.
    - Możliwe wartości: Dowolna liczba rzeczywista.


5. **n_neighbor**
    - Opis: Wartość k w k-najbliższych sąsiadów.
    - Możliwe wartości: Dowolna liczba całkowita.


6. **kernel_function**
    - Opis: Funkcja kernela odpowiadająca SVM.
    - Możliwe wartości: Zazwyczaj 'rbf', 'linear' lub dowolna funkcja zdefiniowana przez użytkownika.


7. **gamma_k**
    - Opis: Parametr gamma odpowiadający kernel_function.
    - Możliwe wartości: Dowolna liczba rzeczywista.


8. **gamma_A**
    - Opis: Waga kary za złożoność funkcji.
    - Możliwe wartości: Dowolna liczba rzeczywista.


9. **gamma_I**
    - Opis: Waga kary za gładkość rozkładu danych.
    - Możliwe wartości: Dowolna liczba rzeczywista.

In [18]:
from LAMDA_SSL.Algorithm.Classification.LapSVM import LapSVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
#datasets = ['cifar10','imdb', 'breast_cancer']
#datasets = ['imdb', 'breast_cancer']
datasets = ['breast_cancer']
labeled_size = 0.3
models = {dataset: LapSVM() for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
breast_cancer Results: {'accuracy': 0.6228070175438597, 'precision': 0.6228070175438597, 'recall': 1.0, 'f1_score': 0.7675675675675676}
Training Time breast_cancer: 0.09562420845031738



# <font color='green'>Ladder networks</font>

# <font color='green'>$\Pi$-model</font>

# <font color='green'>Temporal ensembling</font>

### Hiperparametry TemporalEnsembling

- `lambda_u` (waga straty nienadzorowanej): 
  - Możliwe wartości: dowolna liczba zmiennoprzecinkowa.

- `ema_weight` (waga aktualizacji dla eksponencjalnie ważonej średniej ruchomej pseudoetykiet):
  - Możliwe wartości: dowolna liczba zmiennoprzecinkowa między 0 a 1.

- `warmup` (koniec okresu rozgrzewki):
  - Możliwe wartości: procent całkowitej liczby iteracji, np. 0.4 oznacza 40% całkowitej liczby iteracji.

- `num_classes` (liczba klas):
  - Możliwe wartości: dowolna liczba całkowita.

- `num_samples` (liczba próbek):
  - Możliwe wartości: dowolna liczba całkowita.

# <font color='green'>Mean teacher</font>

### Hiperparametry

- **lambda_u**
    - Waga straty nienadzorowanej. Możliwe wartości zależą od specyfiki zadania, ale często wybiera się wartości z zakresu 0.1 do 1.

- **warmup**
    - Końcowy punkt okresu rozgrzewki. Jeśli num_it_total wynosi 100, a warmup jest ustawione na 0.4, to rozgrzewka odbywa się w pierwszych 40 iteracjach. Możliwe wartości to liczby z zakresu od 0 do 1.

- **ema_decay**
    - Współczynnik zaniku dla eksponencjalnego średniego ruchomego. Typowe wartości to liczby z zakresu od 0.9 do 0.999.

- **weight_decay**
    - Współczynnik regularyzacji, który pomaga zapobiegać przeuczeniu. Zwykle wybiera się wartości od 0.0001 do 0.01.

# <font color='green'>VAT Virtual Adversarial Training</font>

### Hiperparametry

- **lambda_u** (waga straty nienadzorowanej):
    - Możliwe wartości: liczby zmiennoprzecinkowe, zazwyczaj w zakresie od 0 do 1.


- **eps** (poziom szumu):
    - Możliwe wartości: małe liczby zmiennoprzecinkowe, zazwyczaj poniżej 1.


- **warmup** (koniec okresu rozgrzewki):
    - Możliwe wartości: liczby zmiennoprzecinkowe z zakresu od 0 do 1, wskazujące na proporcję liczby iteracji, w których odbywa się rozgrzewka.


- **xi** (parametr skali dla zakłóceń):
    - Możliwe wartości: małe liczby zmiennoprzecinkowe, często bliskie 0.


- **lambda_entmin** (waga straty z minimalizacji entropii):
    - Możliwe wartości: liczby zmiennoprzecinkowe, zwykle w zakresie od 0 do 1.

# Transductive
## <font color='green'>TSVM</font>

### Hiperparametry

- `Cl` (Waga próbek oznaczonych)
    - Dowolna wartość liczby rzeczywistej.


- `Cu` (Waga próbek nieoznaczonych)
    - Dowolna wartość liczby rzeczywistej.


- `kernel` (Typ jądra)
    - Możliwe wartości to 'rbf', 'knn' lub funkcja wywoływalna.


- `degree` (Stopień wielomianu)
    - Dotyczy jądra 'poly', wartości całkowite.


- `gamma` (Parametr gamma)
    - Dotyczy jąder 'rbf', 'poly', 'sigmoid', dowolna wartość liczby rzeczywistej.


- `coef0` (Stała składowa funkcji jądra)
    - Dotyczy jąder 'poly' i 'sigmoid', dowolna wartość liczby rzeczywistej.


- `shrinking` (Użycie heurystyki skurczania)
    - Wartości `True` lub `False`.


- `probability` (Wagi dla klasyfikacji kąta obrotu)
    - Dowolna wartość liczby rzeczywistej.


- `tol` (Tolerancja na zatrzymanie treningu)
    - Dowolna wartość liczby rzeczywistej, domyślnie 1e-3.


- `cache_size` (Rozmiar pamięci podręcznej funkcji jądra)
    - Dowolna wartość liczby rzeczywistej.


- `class_weight` (Wagi różnych klas)
    - Słownik lub 'balanced'.


- `max_iter` (Maksymalna liczba iteracji)
    - Wartość całkowita, `-1` dla nieograniczonej liczby.


- `decision_function_shape` (Kształt funkcji decyzyjnej)
    - 'ovo' lub 'ovr', domyślnie 'ovr'.


- `break_ties` (Decyzja w przypadku remisu)
    - Wartości `True` lub `False`.

In [17]:
from LAMDA_SSL.Algorithm.Classification.TSVM import TSVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Inicjalizacja modeli i danych
#datasets = ['cifar10','imdb', 'breast_cancer']
#datasets = ['imdb', 'breast_cancer']
datasets = ['breast_cancer']
labeled_size = 0.3
models = {dataset: TSVM() for dataset in datasets}
results = {}

# Przetwarzanie danych i trenowanie modeli
for dataset in datasets:
    print(f"Processing {dataset}")
    X_train, y_train, X_test, y_test = preprocess_data(dataset)
    X_labeled, y_labeled, X_unlabeled = split_labeled_unlabeled(X_train, y_train, labeled_size)
    print(f"X_labeled shape: {X_labeled.shape}, X_unlabeled shape: {X_unlabeled.shape}")
    print(f"y_labeled shape: {y_labeled.shape}")
    results[dataset], training_time = train_and_evaluate(dataset, models[dataset], X_labeled, y_labeled, X_unlabeled, X_test, y_test)
    print(f"{dataset} Results:", results[dataset])
    print(f"Training Time {dataset}:", training_time)
    print()


Processing breast_cancer
X_labeled shape: (136, 30), X_unlabeled shape: (319, 30)
y_labeled shape: (136,)
Training model on breast_cancer
X_train shape: (136, 30), X_unlabeled shape: (319, 30)
y_train shape: (136,)
[1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1
 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1
 0 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 0 1
 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 0 1 1 1 0
 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1
 1 1 1 0 0 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 1 1 0
 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0]


ValueError: Found input variables with inconsistent numbers of samples: [114, 319]

# <font color='green'>Label propagation algorithm for inference on graphs</font>

### Hiperparametry

- **kernel**
    - Opis: Funkcja jądra, która może być podana jako ciąg znaków 'rbf' lub 'knn', lub jako funkcja.
    - Możliwe wartości: 'rbf', 'knn', funkcja.


- **gamma**
    - Opis: Wartość gamma, gdy funkcja jądra to jądro RBF.
    - Możliwe wartości: Liczba zmiennoprzecinkowa (float).


- **n_neighbors**
    - Opis: Liczba sąsiadów, gdy funkcja jądra to jądro 'n_neighbors'.
    - Możliwe wartości: Liczba całkowita (int).


- **max_iter**
    - Opis: Maksymalna liczba iteracji.
    - Możliwe wartości: Liczba całkowita (int).


- **tol**
    - Opis: Tolerancja zbieżności.
    - Możliwe wartości: Liczba zmiennoprzecinkowa (float).


- **n_jobs**
    - Opis: Liczba równoległych zadań.
    - Możliwe wartości: Liczba całkowita (int) lub None.